In [19]:
import os
import settings
import datetime
import pandas as pd
import numpy as np
import clean

In [8]:
df = pd.read_csv("data/LoanStats_2016Q1.csv", header = 1, )

/Users/chitwan/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
df.describe()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,url,dti,delinq_2yrs,inq_last_6mths,...,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_il_6m,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog
count,0.0,133887.000000,133887.000000,133887.000000,133887.000000,1.338870e+05,0.0,133887.000000,133887.000000,133886.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,15589.394041,15589.394041,15581.984023,460.870707,8.046397e+04,NaN,20.870784,0.343476,0.576498,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,8858.198349,8858.198349,8856.454888,261.616439,7.808478e+04,NaN,115.252571,0.908581,0.878987,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,1000.000000,1000.000000,775.000000,30.120000,0.000000e+00,NaN,-1.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,8400.000000,8400.000000,8400.000000,265.680000,4.950000e+04,NaN,12.640000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,14400.000000,14400.000000,14400.000000,398.520000,6.800000e+04,NaN,18.760000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,21000.000000,21000.000000,21000.000000,613.910000,9.500800e+04,NaN,25.770000,0.000000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,40000.000000,40000.000000,40000.000000,1536.950000,9.550000e+06,NaN,9999.000000,22.000000,5.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
files = os.listdir(settings.DATA_DIR)
files

['.DS_Store',
 'LoanStats_2016Q3.csv',
 'LoanStats_2016Q2.csv',
 'LoanStats_2016Q1.csv']

In [15]:
files = []

for item in os.listdir(settings.DATA_DIR):
    if item.endswith(".csv"):
        files.append(item)

In [17]:
files

['LoanStats_2016Q3.csv', 'LoanStats_2016Q2.csv', 'LoanStats_2016Q1.csv']

In [2]:
SELECT = [
"loan_amnt",
"term",
"int_rate",
"installment",
"grade",
"sub_grade",
"emp_title",
"emp_length",
"home_ownership",
"annual_inc",
"verification_status",
"issue_d",
"loan_status",
"purpose",
"addr_state",
"dti",
"earliest_cr_line",
"open_acc",
"pub_rec",
"revol_util",
"total_acc",
"initial_list_status",
"application_type"
]

dt = {
"loan_amnt": float,
"term": object,
"int_rate": object,
"installment": float,
"grade": object,
"sub_grade": object,
"emp_title": object,
"emp_length": object,
"home_ownership": object,
"annual_inc": float,
"verification_status": object,
"issue_d": object,
"loan_status": object,
"purpose": object,
"addr_state": object,
"dti": float,
"earliest_cr_line": object,
"open_acc": float,
"pub_rec": float,
"revol_util": object,
"total_acc": float,
"initial_list_status": object,
"application_type": object
}

In [99]:
def p2f(x):
    return float(x.strip().strip('%'))/100


In [4]:
raw12_13= pd.read_csv("my analysis/LoanStats3b.csv", header = 1, dtype=dt, usecols = SELECT, na_values = [""] )
raw12_13.drop(raw12_13.tail(3).index, inplace = True)


In [6]:
raw12_13.to_pickle('my analysis/data/12-13raw.pkl')

In [5]:
raw12_13.loan_status[raw12_13.loan_status.isin(["Current"])].count()

7955

In [6]:
def lookup(s):
    """
    This is an extremely fast approach to datetime parsing.
    For large data, the same dates are often repeated. Rather than
    re-parse these, we store all unique dates, parse them, and
    use a lookup to convert all dates.
    """
    dates = {date:pd.to_datetime(date) for date in s.unique()}
    return s.map(dates)


In [7]:
for column in ["issue_d", "earliest_cr_line"]:
        # Convert date columns into datetime objects using the lookup function
        alldata[column] = lookup(alldata[column])
        
        # Create month and year columns for each date column and delete the original date column
        alldata["{}_year".format(column)] = pd.to_numeric(alldata[column].dt.year)
        alldata["{}_month".format(column)] = pd.to_numeric(alldata[column].dt.month)
        del alldata[column]

In [9]:
today = pd.to_datetime('today')
curr_quart = (today.month-1)//3 + 1
curr_year = today.year
issue_quart = (alldata["issue_d_month"]-1)//3 + 1
issue_year = alldata["issue_d_year"]
alldata["tracked_quart"] = (curr_year - issue_year)*4 + (curr_quart - issue_quart) - 1

In [12]:
alldata.tracked_quart.head()

0    16
1    16
2    16
3    16
4    16
Name: tracked_quart, dtype: int64

In [13]:
alldata.issue_d_month.head()

0    12
1    12
2    12
3    12
4    12
Name: issue_d_month, dtype: int64

In [14]:
alldata.issue_d_year.head()

0    2013
1    2013
2    2013
3    2013
4    2013
Name: issue_d_year, dtype: int64

In [63]:
((raw12_13.loan_status.value_counts())["Fully Paid"] + (raw12_13.loan_status.value_counts())["Current"])/ raw12_13.loan_status.count()

0.84246466149431398

In [45]:
raw12_13["int_rate"]

0           8.90%
1          13.53%
2          13.53%
3          12.85%
4          11.99%
5          10.99%
6          13.53%
7           9.67%
8           6.62%
9          12.85%
10         22.90%
11          7.62%
12         10.99%
13         10.99%
14         16.24%
15         19.97%
16         10.99%
17          7.62%
18         14.47%
19         13.98%
20         14.98%
21         20.50%
22         11.99%
23          7.62%
24         15.61%
25          9.67%
26          7.90%
27         10.99%
28         18.25%
29          6.03%
           ...   
188150     15.27%
188151     17.27%
188152     17.58%
188153     16.29%
188154     11.71%
188155      6.03%
188156     13.49%
188157      6.03%
188158     12.42%
188159     12.42%
188160     17.58%
188161     14.65%
188162      6.03%
188163     15.96%
188164     14.65%
188165     22.35%
188166      9.91%
188167     22.74%
188168     19.91%
188169     17.58%
188170     22.74%
188171     14.27%
188172     12.69%
188173     14.65%
188174    

In [42]:
y = pd.Series([float(x.strip().strip('%'))/100 for x in raw12_13.int_rate])
y.head()

0    0.0890
1    0.1353
2    0.1353
3    0.1285
4    0.1199
dtype: float64

In [7]:
raw14= pd.read_csv("my analysis/LoanStats3c.csv", header = 1, dtype=dt, usecols = SELECT, na_values = [""] )
raw14.drop(raw14.tail(3).index, inplace = True)

In [8]:
raw14.to_pickle('my analysis/data/14raw.pkl')

In [39]:
((raw14.loan_status.value_counts())["Fully Paid"] + (raw14.loan_status.value_counts())["Current"])/ raw14.loan_status.count()

0.82890403517408795

In [9]:
raw15= pd.read_csv("my analysis/LoanStats3d.csv", header = 1, dtype=dt, usecols = SELECT, na_values = [""] )
raw15.drop(raw15.tail(3).index, inplace = True)

In [10]:
raw15.to_pickle('my analysis/data/15raw.pkl')

In [40]:
((raw15.loan_status.value_counts())["Fully Paid"] + (raw15.loan_status.value_counts())["Current"])/ raw15.loan_status.count()

0.83537879903299506

In [11]:
rawQ1= pd.read_csv("my analysis/LoanStats_2016Q1.csv", header = 1, dtype=dt, usecols = SELECT, na_values = [""] )
rawQ1.drop(rawQ1.tail(3).index, inplace = True)

In [12]:
rawQ1.to_pickle('my analysis/data/16Q1raw.pkl')

In [41]:
((rawQ1.loan_status.value_counts())["Fully Paid"] + (rawQ1.loan_status.value_counts())["Current"])/ rawQ1.loan_status.count()

0.9063008828406256

In [13]:
rawQ2= pd.read_csv("my analysis/LoanStats_2016Q2.csv", header = 1, dtype=dt, usecols = SELECT, na_values = [""] )
rawQ2.drop(rawQ2.tail(3).index, inplace = True)

In [14]:
rawQ2.to_pickle('my analysis/data/16Q2raw.pkl')

In [42]:
((rawQ2.loan_status.value_counts())["Fully Paid"] + (rawQ2.loan_status.value_counts())["Current"]) / rawQ2.loan_status.count()

0.9238142928678732

In [15]:
rawQ3= pd.read_csv("my analysis/LoanStats_2016Q3.csv", header = 1, dtype=dt, usecols = SELECT, na_values = [""] )
rawQ3.drop(rawQ3.tail(3).index, inplace = True)

In [16]:
rawQ3.to_pickle('my analysis/data/16Q3raw.pkl')

In [43]:
((rawQ3.loan_status.value_counts())["Fully Paid"] + (rawQ3.loan_status.value_counts())["Current"]) / rawQ3.loan_status.count()

0.94455149870357857

In [20]:
rawQ4= pd.read_csv("my analysis/LoanStats_2016Q4.csv", header = 1, dtype=dt, usecols = SELECT, na_values = [""] )
rawQ4.drop(rawQ4.tail(3).index, inplace = True)

In [21]:
rawQ4.to_pickle('my analysis/data/16Q4raw.pkl')

In [22]:
raw17Q1= pd.read_csv("my analysis/LoanStats_2017Q1.csv", header = 1, dtype=dt, usecols = SELECT, na_values = [""] )
raw17Q1.drop(raw17Q1.tail(3).index, inplace = True)

In [23]:
raw17Q1.to_pickle('my analysis/data/17Q1raw.pkl')

In [24]:
raw17Q2= pd.read_csv("my analysis/LoanStats_2017Q2.csv", header = 1, dtype=dt, usecols = SELECT, na_values = [""] )
raw17Q2.drop(raw17Q2.tail(3).index, inplace = True)

In [25]:
raw17Q2.to_pickle('my analysis/data/17Q2raw.pkl')

In [121]:
df.loan_status.unique()

array(['Current', 'Fully Paid', 'Late (31-120 days)', 'Charged Off',
       'In Grace Period', 'Late (16-30 days)', 'Default'], dtype=object)

In [27]:
df = pd.read_csv("processed/alldata.csv", header = 0)

In [28]:
df.shape

(330856, 23)

In [29]:
df.loan_status.value_counts() / df.loan_status.count()

Current               0.777894
Fully Paid            0.145045
Charged Off           0.039779
Late (31-120 days)    0.021768
In Grace Period       0.009917
Late (16-30 days)     0.005567
Default               0.000030
Name: loan_status, dtype: float64

In [2]:
dat = pd.read_pickle('processed/assembled.pkl')

In [3]:
clean.int_rate2float(dat)
clean.target2binary(dat)
clean.categorical_code(dat)

In [6]:
def split_dates(alldata):
    """ 
    Splits dates into month and year column and deletes the 
    original date column. Function also eliminates loans that
    are less than 1.5 years old (see main.ipynb for complete reasoning)
    """
    def lookup(s):
        """
        This is an extremely fast approach to datetime parsing.
        For large data, the same dates are often repeated. Rather than
        re-parse these, we store all unique dates, parse them, and
        use a lookup to convert all dates.
        """
        dates = {date:pd.to_datetime(date) for date in s.unique()}
        return s.map(dates)

    for column in ["issue_d", "earliest_cr_line"]:
        # Convert date columns into datetime objects using the lookup function
        alldata[column] = lookup(alldata[column])
        
        # Create month and year columns for each date column and delete the original date column
        alldata["{}_year".format(column)] = pd.to_numeric(alldata[column].dt.year)
        alldata["{}_month".format(column)] = pd.to_numeric(alldata[column].dt.month)
        del alldata[column]




In [7]:
split_dates(dat)

In [10]:
# Keep only data that meets the minimum tracking quarters requirement (defined in settings.py)
today = pd.to_datetime('today')
curr_quart = (today.month-1)//3 + 1
curr_year = today.year


2017

In [23]:
issue_quart = (dat["issue_d_month"]-1)//3 + 1
issue_year = dat["issue_d_year"]
issue_year

0          2016.0
1          2016.0
2          2016.0
3          2016.0
4          2016.0
5          2016.0
6          2016.0
7          2016.0
8          2016.0
9          2016.0
10         2016.0
11         2016.0
12         2016.0
13         2016.0
14         2016.0
15         2016.0
16         2016.0
17         2016.0
18         2016.0
19         2016.0
20         2016.0
21         2016.0
22         2016.0
23         2016.0
24         2016.0
25         2016.0
26         2016.0
27         2016.0
28         2016.0
29         2016.0
            ...  
1218271    2007.0
1218272    2007.0
1218273    2007.0
1218274    2007.0
1218275    2007.0
1218276    2007.0
1218277    2007.0
1218278    2007.0
1218279    2007.0
1218280    2007.0
1218281    2007.0
1218282    2007.0
1218283    2007.0
1218284    2007.0
1218285    2007.0
1218286    2007.0
1218287    2007.0
1218288    2007.0
1218289    2007.0
1218290    2007.0
1218291    2007.0
1218292    2007.0
1218293    2007.0
1218294    2007.0
1218295   

In [39]:
dat["tracked_quart"] = (curr_year - issue_year)*4 + (curr_quart - issue_quart) - 1

dat = dat[dat["tracked_quart"] > settings.MINIMUM_TRACKING_QUARTERS]

dat



(1021326, 25)

In [43]:
dat = dat.fillna(-1)
dat.shape

(1021326, 25)

In [49]:
cleaned = pd.read_pickle("processed/cleaned.pkl")

In [50]:
cleaned

,loan_amnt,term,int_rate,installment,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,...,revol_util,initial_list_status,application_type,tot_cur_bal,tot_hi_cred_lim,issue_d_year,issue_d_month,earliest_cr_line_year,earliest_cr_line_month,tracked_quart
0,16000.0,1,9.75,337.99,7,1,4,120000.0,1,True,...,30.5,1,1,109019.0,160515.0,2016.0,3.0,1994.0,11.0,6.0
1,6000.0,0,7.39,186.34,3,3,1,72000.0,1,True,...,23.6,1,1,24618.0,45139.0,2016.0,3.0,2008.0,10.0,6.0
2,5625.0,0,18.25,204.07,17,1,1,53328.0,2,True,...,26.9,0,1,156318.0,200449.0,2016.0,3.0,2005.0,3.0,6.0
3,6000.0,0,10.75,195.73,8,8,5,35000.0,1,True,...,75.3,1,1,26867.0,33626.0,2016.0,3.0,2005.0,5.0,6.0
4,10000.0,0,8.39,315.17,5,10,5,48000.0,1,True,...,52.1,1,1,39061.0,61807.0,2016.0,3.0,2012.0,3.0,6.0
5,6000.0,0,7.39,186.34,3,3,1,72000.0,1,True,...,23.6,1,1,24618.0,45139.0,2016.0,3.0,2008.0,10.0,6.0
6,5625.0,0,18.25,204.07,17,1,1,53328.0,2,True,...,26.9,0,1,156318.0,200449.0,2016.0,3.0,2005.0,3.0,6.0
7,6000.0,0,10.75,195.73,8,8,5,35000.0,1,True,...,75.3,1,1,26867.0,33626.0,2016.0,3.0,2005.0,5.0,6.0
8,10000.0,0,8.39,315.17,5,10,5,48000.0,1,True,...,52.1,1,1,39061.0,61807.0,2016.0,3.0,2012.0,3.0,6.0
9,17000.0,0,9.75,546.55,7,10,1,40000.0,2,True,...,85.9,1,1,31939.0,44131.0,2016.0,3.0,2006.0,8.0,6.0


Things we need to do: 
- make the target var (loan_status) into 0/1 (fully paid off or not); throw out current loans
- turn all categorical variables into numerics
- parse in the dates as actual dates?
- split dates into month and year columns
